<a href="https://colab.research.google.com/github/rklepov/hse-cs-ml-2018-2019/blob/homework/09-NLP/HW/hw3/nlp_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [*HW_texts*](https://github.com/rklepov/hse-cs-ml-2018-2019/blob/homework/09-NLP/HW/HW_texts.ipynb "HW_texts.ipynb") / Задание 2

---


In [0]:
import pickle
import os

import warnings
[ warnings.simplefilter(action='ignore', category=w) for w in [UserWarning, DeprecationWarning, FutureWarning] ];

In [2]:
import string
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import WhitespaceTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import gensim

from gensim.models.word2vec import Word2Vec
from gensim.models.phrases import Phrases, Phraser

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Корпус текстов

Загрузка корпуса текстов (новости с сайта РИА-новости):

In [5]:
!ls -hl data/

with open('data/ria_news_2018.pickle', 'rb') as f:
    ria_news = pickle.load(f)

total 893M
-rw------- 1 root root 893M Sep 30 10:02 ria_news_2018.pickle


### Подготовка текста



*   Очистка от пунктуации (используем регулярное выражение)
*   Приведение к нижнему регистру
*   Токенизация (просто по пробельным символам)
*   _Без лемматизации_ (большой корпус)
*   Удаление стоп-слов





In [0]:
def make_punct_remover(pattern = r'[^\w]+'):
    regex = re.compile(pattern)
    return lambda text : regex.sub(' ', text)

In [0]:
def make_prepare_text(punct_remover, tokenizer, lemmatizer, stopwords):
    def prepare_text(text):
        text = punct_remover(text)    
        words = tokenizer.tokenize(text.lower())
        words = lemmatizer(words)
        words = [ w for w in words if len(w) > 1 and w not in stopwords ]
        return words
    return prepare_text

In [0]:
stop_words_ru = [ item for item in stopwords.words('russian') ]
stop_words_en = stopwords.words('english') 

stop_words = frozenset(stop_words_ru + stop_words_en)

Функция подготовки текста:

In [0]:
prepare_text = make_prepare_text(make_punct_remover(),
                                 WhitespaceTokenizer(),
                                 lambda x: x, # trivial "lemmatizer"
                                 stop_words)                               

### Генератор документов

Объединяем заголовок и текст новости, применяем препроцессинг и возвращаем итоговый список слов.

In [0]:
def doc_gen(corpus, prepare_text):
    for doc in corpus:
        text = ' '.join([doc['title'], doc['text']])
        yield prepare_text(text)

Поскольку по документам корпуса необходимо будет проходить несколько раз (**n** раз при выделении **n**-грамм и потом ещё раз при обучении модели), то, чтобы не повторять препроцессинг много раз, однократно применим его к текстам и сохраним результат (в список списков слов). В данном случае объём памяти позволяет нам это сделать. В противном случае пришлось бы искать иные варианты: либо согласиться на многократное повторение предобработки в генераторе текстов (который, в свою очередь, пришлось бы создавать несколько раз), либо пройтись предобработкой по текстам один раз, сохраняя результаты для каждого текста в отдельный файл в общей для корпуса текстов директории. Во втором случае потом потребовалось бы использовать другой генератор, который бы умел подгружать тексты из файлов (см. [`glob`](https://docs.python.org/3/library/glob.html "glob — Unix style pathname pattern expansion &#8212; Python documentation") и т.д.).

In [0]:
ria_news_corpus = [ text for text in doc_gen(ria_news, prepare_text) ]

Оригинальный список новостей из `ria_news_2018.pickle` нам больше не потребуется, поэтому для освобождения памяти его можно удалить.

In [0]:
del ria_news

Генератор **n**-грам (для выбранного **n**) на основе [`gensim Phraser`](https://radimrehurek.com/gensim/models/phrases.html "gensim: models.phrases – Phrase (collocation) detection"):

In [0]:
def apply_transformers(transformers, doc):
    if 0 == len(transformers):
        return doc
    return apply_transformers(transformers[1:], transformers[0][doc])

def ngram_generator(corpus, transformers):
    for text in corpus:
        yield apply_transformers(transformers, [word for word in text])

def make_ngram_gen(make_corpus, ngram_gen, transformers, ngram, **kwargs):
    if 1 >= ngram:
        return ngram_gen, transformers
    transformers += [ Phraser(Phrases(ngram_gen)) ]
    generator = ngram_generator(make_corpus(**kwargs), transformers)
    return make_ngram_gen(make_corpus, generator, transformers, ngram - 1, **kwargs)

def make_ngram_gen_and_save(root_dir, pre, make_uni_gen, ngram):
    ngram_gen, ngram_trans = make_ngram_gen(make_uni_gen, make_uni_gen(), [], ngram)
    for n, transformer in enumerate(ngram_trans):
        transformer.save(os.path.join(root_dir, f'{pre}_{n + 2}gram.transformer'))
    return ngram_gen, ngram_trans

Создадим генератор *би*грамм и сохраним построителя *би*грамм ([`Phraser`](https://radimrehurek.com/gensim/models/phrases.html "gensim: models.phrases – Phrase (collocation) detection")) для дальнейшей возможности переиспользования, потому что обучается он довольно долго.

In [0]:
make_uni_gen = lambda: (doc for doc in ria_news_corpus)

bigram_trans_path = 'model/ria_news_2gram.transformer'

!mkdir -p model/

if not os.path.exists(bigram_trans_path):
    bigram_gen, bigram_trans = make_ngram_gen_and_save('model', 'ria_news', make_uni_gen, 2)
else:
    bigram_trans = [ Phraser.load(bigram_trans_path) ]
    bigram_gen = ngram_generator(make_uni_gen(), bigram_trans)

### Модель Word2Vec

Если ещё нет ранее обученной и сохранённой модели, то сначала выполняем построение словаря, а потом обучаем модель (перед вторым действием повторно создаём генератор, потому что предыдущий уже закончил итерирование).

In [15]:
%%time

w2v_model_path = 'model/ria_news_w2v_2gram_300.model'

!mkdir -p model/

if not os.path.exists(w2v_model_path):
    w2v_model = Word2Vec(size=300, window=7, min_count=10, workers=8)
    w2v_model.build_vocab(bigram_gen)
    bigram_gen = ngram_generator(make_uni_gen(), bigram_trans)
    w2v_model.train(bigram_gen, total_examples=w2v_model.corpus_count, epochs=20)
    w2v_model.save(w2v_model_path)
else:
    w2v_model = Word2Vec.load(w2v_model_path)

print(w2v_model)

Word2Vec(vocab=231435, size=300, alpha=0.025)
CPU times: user 6min 16s, sys: 2.06 s, total: 6min 18s
Wall time: 4min 21s


## Свойства модели

In [16]:
w2v_model.wv.most_similar('москва')[:5]

[('волгоград_екатеринбург', 0.7432365417480469),
 ('британских_дипломата', 0.7392398118972778),
 ('категорически_отвергает', 0.7117505669593811),
 ('казань_калининград', 0.7116988301277161),
 ('неоднократно_высказывала', 0.693545937538147)]

### Арифметика

In [17]:
w2v_model.most_similar(positive=['москва', 'франция'], negative=['россия'])[:5]

[('британских_дипломата', 0.804212212562561),
 ('лондон_париж', 0.7469882369041443),
 ('российских_дипломата', 0.7165673971176147),
 ('нидерланды', 0.7158944606781006),
 ('чехия', 0.7157461643218994)]

In [18]:
w2v_model.most_similar(positive=['москва', 'екатеринбург'], negative=['собянин'])[:5]

[('новатор_московская', 0.7216000556945801),
 ('ростов_дону', 0.7214747667312622),
 ('екатеринбург_казань', 0.6864383220672607),
 ('дельта_саратов', 0.6818997859954834),
 ('сити_ленинградская', 0.6740564107894897)]

In [19]:
w2v_model.most_similar(positive=['спартак', 'санкт_петербург'], negative=['москва'])[:5]

[('первой_домашней', 0.9051880836486816),
 ('домашние_матчи', 0.8987385034561157),
 ('торпедо', 0.8929771780967712),
 ('арене', 0.8884447813034058),
 ('смолов_перешел', 0.8858283758163452)]

In [20]:
w2v_model.most_similar(positive=['иркутск', 'сочи'], negative=['байкал'])[:5]

[('якутск_сен', 0.7628297209739685),
 ('фоторабот', 0.7478163242340088),
 ('соцвыплат', 0.7466264963150024),
 ('28_июл', 0.7453728318214417),
 ('венеция_сен', 0.7417500019073486)]

In [21]:
w2v_model.most_similar(positive=['чай', 'кофе'], negative=['лимон'])[:5]

[('погружаться', 0.9500764608383179),
 ('устроен', 0.949739396572113),
 ('окружающую', 0.9477403163909912),
 ('резко_изменилась', 0.9466489553451538),
 ('успеваешь', 0.9459283947944641)]

In [22]:
w2v_model.most_similar(positive=['авиакомпания', 'ржд'], negative=['аэрофлот'])[:5]

[('экспертизу', 0.8235124945640564),
 ('сотрудница', 0.8220018148422241),
 ('регистрации_транспортных', 0.8209173679351807),
 ('администрация', 0.8182415962219238),
 ('проверило', 0.8159729838371277)]

### Найтии лишнее

In [23]:
 w2v_model.doesnt_match(['магазин', 'супермаркет', 'рынок', 'тц'])

'рынок'

In [24]:
w2v_model.doesnt_match(['теннис', 'хоккей', 'футбол', 'дзюдо'])

'дзюдо'

In [25]:
w2v_model.doesnt_match(['кошка', 'собака', 'попугай', 'кролик'])

'кролик'

In [26]:
w2v_model.doesnt_match(['коми', 'дагестан', 'башкирия', 'камчатка'])

'коми'

## Координаты

***TODO***

## Оценка качества

[RusVectōrēs: модели](https://rusvectores.org/ru/models "RusVectōrēs: семантические модели для русского языка")

***TODO***

## References

[1] [Gensim Tutorial – A Complete Beginners Guide](https://www.machinelearningplus.com/nlp/gensim-tutorial "Gensim Tutorial - A Complete Beginners Guide – Machine Learning Plus")

[2] [RaRe-Technologies/gensim/word2vec.ipynb: *Word2Vec Tutorial*](https://github.com/RaRe-Technologies/gensim/blob/master/docs/notebooks/word2vec.ipynb "gensim/word2vec.ipynb at master · RaRe-Technologies/gensim")

[3] [gensim: Tutorials](https://radimrehurek.com/gensim/tutorial.html "gensim: Tutorials")

[4] [gensim: models.phrases – Phrase (collocation) detection](https://radimrehurek.com/gensim/models/phrases.html "gensim: models.phrases – Phrase (collocation) detection")

[5] [gensim: models.word2vec – Word2vec embeddings](https://radimrehurek.com/gensim/models/word2vec.html "gensim: models.word2vec – Word2vec embeddings")

[6] [bhargavvader/personal/word2vec.ipynb: *Word2Vec*](https://github.com/bhargavvader/personal/blob/master/notebooks/text_analysis/word2vec.ipynb "personal/word2vec.ipynb at master · bhargavvader/personal")